In [1]:
from torchtext import data
from torchtext.data import BucketIterator

def read_data(input_file, max_length):
    with open(input_file, encoding="utf8") as f:
        poetries = []
        poetry = []
        for line in f:
            contends = line.strip()
            if len(poetry) + len(contends) <= max_length:
                if contends:
                    poetry.extend(contends)
                else:
                    poetries.append(poetry)
                    poetry = []
            else:
                poetries.append(poetry)
                poetry = list(contends)
        if poetry:
            poetries.append(poetry)
        return poetries


class PoetryDataset(data.Dataset):

    def __init__(self, text_field, datafile, max_length, **kwargs):
        fields = [("text", text_field)]
        datas = read_data(datafile, max_length)
        examples = []
        for text in datas:
            examples.append(data.Example.fromlist([text], fields))
        super(PoetryDataset, self).__init__(examples, fields, **kwargs)


def load_iters(eos_token="[EOS]", batch_size=32, device="cpu", data_path='data', max_length=128):
    TEXT = data.Field(eos_token=eos_token, batch_first=True, include_lengths=True)
    datas = PoetryDataset(TEXT, "./poetryFromTang.txt", max_length)
    train_data, dev_data, test_data = datas.split([0.8, 0.1, 0.1])

    TEXT.build_vocab(train_data)

    train_iter, dev_iter, test_iter = BucketIterator.splits(
        (train_data, dev_data, test_data),
        batch_sizes=(batch_size, batch_size, batch_size),
        device=device,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        repeat=False,
        shuffle=True
    )
    return train_iter, dev_iter, test_iter, TEXT

In [8]:
# -*- coding:utf8 -*-
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm, trange
from tensorboardX import SummaryWriter
import math

torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64
HIDDEN_DIM = 512
LAYER_NUM = 1
EPOCHS = 200
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.01
MOMENTUM = 0.9
CLIP = 5
DECAY_RATE = 0.05  # learning rate decay rate
EOS_TOKEN = "[EOS]"
DATA_PATH = 'data'
EMBEDDING_SIZE = 200
TEMPERATURE = 0.8  # Higher temperature means more diversity.
MAX_LEN = 64


def train(train_iter, dev_iter, loss_func, optimizer, epochs, clip):
    for epoch in trange(epochs):
        model.train()
        total_loss = 0
        total_words = 0
        for i, batch in enumerate(tqdm(train_iter)):
            text, lens = batch.text
            if epoch == 0 and i == 0:
                tqdm.write(' '.join([TEXT.vocab.itos[i] for i in text[0]]))
                tqdm.write(' '.join([str(i.item()) for i in text[0]]))
            inputs = text[:, :-1]
            targets = text[:, 1:]
            init_hidden = model.lstm.init_hidden(inputs.size(0))
            logits, _ = model(inputs, lens - 1, init_hidden)  # [EOS] is included in length.
            loss = loss_func(logits.reshape(-1, logits.size(-1)), targets.reshape(-1))

            model.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            total_loss += loss.item()
            total_words += lens.sum().item()
        tqdm.write("Epoch: %d, Train perplexity: %d" % (epoch + 1, math.exp(total_loss / total_words)))
        writer.add_scalar('Train_Loss', total_loss, epoch)
        eval(dev_iter, True, epoch)

        lr = LEARNING_RATE / (1 + DECAY_RATE * (epoch + 1))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr


def eval(data_iter, is_dev=False, epoch=None):
    model.eval()
    with torch.no_grad():
        total_words = 0
        total_loss = 0
        for i, batch in enumerate(data_iter):
            text, lens = batch.text
            inputs = text[:, :-1]
            targets = text[:, 1:]
            model.zero_grad()
            init_hidden = model.lstm.init_hidden(inputs.size(0))
            logits, _ = model(inputs, lens - 1, init_hidden)  # [EOS] is included in length.
            loss = loss_func(logits.reshape(-1, logits.size(-1)), targets.reshape(-1))

            total_loss += loss.item()
            total_words += lens.sum().item()
    if epoch is not None:
        tqdm.write(
            "Epoch: %d, %s perplexity %.3f" % (
                epoch + 1, "Dev" if is_dev else "Test", math.exp(total_loss / total_words)))
        writer.add_scalar('Dev_Loss', total_loss, epoch)
    else:
        tqdm.write(
            "%s perplexity %.3f" % ("Dev" if is_dev else "Test", math.exp(total_loss / total_words)))


def generate(eos_idx, word, temperature=0.8):
    model.eval()
    with torch.no_grad():
        if word in TEXT.vocab.stoi:
            idx = TEXT.vocab.stoi[word]
            inputs = torch.tensor([idx])
        else:
            print("%s is not in vocabulary, choose by random." % word)
            prob = torch.ones(len(TEXT.vocab.stoi))
            inputs = torch.multinomial(prob, 1)
            idx = inputs[0].item()

        inputs = inputs.unsqueeze(1).to(device)  # shape [1, 1]
        lens = torch.tensor([1]).to(device)
        hidden = tuple([h.to(device) for h in model.lstm.init_hidden(1)])
        poetry = [TEXT.vocab.itos[idx]]

        while idx != eos_idx:
            logits, hidden = model(inputs, lens, hidden)  # logits: (1, 1, vocab_size)
            word_weights = logits.squeeze().div(temperature).exp().cpu()
            idx = torch.multinomial(word_weights, 1)[0].item()
            inputs.fill_(idx)
            poetry.append(TEXT.vocab.itos[idx])
        print("".join(poetry[:-1]))

In [9]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, dropout_rate=0.5, layer_num=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.layer_num = layer_num
        if layer_num == 1:
            self.lstm = nn.LSTM(input_size, hidden_size, layer_num, batch_first=True)
        else:
            self.lstm = nn.LSTM(input_size, hidden_size, layer_num, dropout=dropout_rate, batch_first=True)

        self.init_weights()

    def init_weights(self):
        for p in self.lstm.parameters():
            if p.dim() > 1:
                nn.init.xavier_normal_(p)
            else:
                p.data.zero_()

    def init_hidden(self, batch_size):
        weight = next(self.parameters())
        return (weight.new_zeros(self.layer_num, batch_size, self.hidden_size),
                weight.new_zeros(self.layer_num, batch_size, self.hidden_size))

    def forward(self, x, lens, hidden):
        '''
        :param x: (batch, seq_len, input_size)
        :param lens: (batch, ), in descending order
        :param hidden: tuple(h,c), each has shape (num_layer, batch, hidden_size)
        :return: output: (batch, seq_len, hidden_size)
                 tuple(h,c): each has shape (num_layer, batch, hidden_size)
        '''
        packed_x = nn.utils.rnn.pack_padded_sequence(x, lens, batch_first=True)
        packed_output, (h, c) = self.lstm(packed_x, hidden)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        return output, (h, c)


class LSTM_LM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size=128, dropout_rate=0.2, layer_num=1, max_seq_len=128):
        super(LSTM_LM, self).__init__()
        self.hidden_size = hidden_size
        self.layer_num = layer_num
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = LSTM(embed_size, hidden_size, dropout_rate, layer_num)
        self.project = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_normal_(self.embed.weight)
        nn.init.xavier_normal_(self.project.weight)

    def forward(self, x, lens, hidden):
        '''
        :param x: (batch, seq_len, input_size)
        :param lens: (batch, ), in descending order
        :param hidden: tuple(h,c), each has shape (num_layer, batch, hidden_size)
        :return: output: (batch, seq_len, hidden_size)
                 tuple(h,c): each has shape (num_layer, batch, hidden_size)
        '''
        embed = self.embed(x)
        hidden, (h, c) = self.lstm(self.dropout(embed), lens, hidden)  # (batch, seq_len, hidden_size)
        out = self.project(self.dropout(hidden))  # (batch, seq_len, vocab_size)
        return out, (h, c)


In [10]:
train_iter, dev_iter, test_iter, TEXT = load_iters(EOS_TOKEN, BATCH_SIZE, device, DATA_PATH, MAX_LEN)
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
eos_idx = TEXT.vocab.stoi[EOS_TOKEN]
model = LSTM_LM(len(TEXT.vocab), EMBEDDING_SIZE, HIDDEN_DIM, DROPOUT_RATE, LAYER_NUM).to(device)

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
loss_func = torch.nn.CrossEntropyLoss(ignore_index=pad_idx, reduction="sum")
writer = SummaryWriter("logs")
train(train_iter, dev_iter, loss_func, optimizer, EPOCHS, CLIP)
eval(test_iter, is_dev=False)
try:
    while True:
        word = input("Input the first word or press Ctrl-C to exit: ")
        generate(eos_idx, word.strip(), TEMPERATURE)
except:
    pass

  0%|          | 0/200 [00:00<?, ?it/s]
                                       


道 傍 过 者 问 行 人 ， 行 人 但 云 点 行 频 。 或 从 十 五 北 防 河 ， 便 至 四 十 西 营 田 。 去 时 里 正 与 裹 头 ， 归 来 头 白 还 戍 边 。 边 亭 流 血 成 海 水 ， 武 皇 开 边 意 未 已 。 [EOS]


  0%|          | 0/200 [00:00<?, ?it/s]
                                       


61 1046 206 180 165 93 6 3 93 6 243 13 1232 93 2272 4 1146 138 104 137 173 2247 158 3 491 146 188 104 94 1316 466 4 47 17 57 555 59 2133 76 3 72 9 76 26 207 1145 205 4 205 212 73 298 117 67 25 3 335 179 66 205 97 28 71 4 2


 20%|██        | 1/5 [00:00<00:03,  1.23it/s]


...

  0%|          | 0/5 [00:00<?, ?it/s]


Epoch: 200, Train perplexity: 427


Epoch: 200, Dev perplexity 448.182


100%|██████████| 200/200 [00:30<00:00,  6.60it/s]


Test perplexity 525.144
Input the first word or press Ctrl-C to exit: 鸟
鸟鼓聚身蝶，海气偃曲离，江寻有清涛渝。华何尘虎纷，中暮中但。渐无溃东。自醒旧交，开风人。论开头臣。未不花，凄歌四，红雪地，九权。毫贵妃我来，万簟锁循屏。梦弦彩边。有衣相，为风自，俗却倾，桃流久。闺日开吾，却受裂四，北山流虚。河海期不，来长席云疏。天道华头白，上识浴来。岁淹不使知，啼帆长梁邴。忽纷难天时何，。
Input the first word or press Ctrl-C to exit: 云
云颓读化生，大阴厥酒但，回流惟热，恨来斡之。石晓日来，之高羁皆，龙暮芙津。风梅一指，上自肯，雄将雪处。岸生相增骅，过日何上华，梦边过。悍掩无犹，秦流与宾。
Input the first word or press Ctrl-C to exit: 水
水商逃得海，自舜主华态。期自高稚奋。首来何鬼，缨陵勇行，宫相飞飞，天收亭。开阳头不，客具五时不。饮尘去家盘，朝开是，人白粲初鲸，何野焉流足火省。
Input the first word or press Ctrl-C to exit: 
 is not in vocabulary, choose by random.
兮人川至家，雾东人花。
Input the first word or press Ctrl-C to exit: 
 is not in vocabulary, choose by random.
觊蒙动父开作。云回马高不，从主浊陵。来亦众乐，曲有。
Input the first word or press Ctrl-C to exit: 
 is not in vocabulary, choose by random.
侍严展云古，牛路长歌，临兵识母，休秋不切，地去有，激知流，归风中。地城马先，家道山，天柳。西但花曲。东白军，手后极，千长都，孙更何，所人白
Input the first word or press Ctrl-C to exit: 
 is not in vocabulary, choose by random.
称策华，连所万，可江云争，泥汝蒯越混。飞腾冥事邑，台白举不荆。靖草河门，天钟中，香清海臣。岸点徒看，沾海翅。远回红衣，几凫，楚来来